In [ ]:
import random
import numpy as np
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objects as go

from scipy.stats import norm, lognorm, truncnorm, gaussian_kde
from plotly.subplots import make_subplots
from scipy.integrate import quad

import math

import matplotlib.pyplot as plt

In [ ]:
import plotly.io as pio

pio.templates.default = "plotly"
pio.templates["plotly"].layout.font.family = "Times New Roman"


In [ ]:
mu    = 2
sigma = 0.5

num_of_samples      = 100
num_of_MCMC_samples = 10000

x = np.linspace(0.01, 100, 10000)

In [ ]:
def p(x):
    return norm.pdf(x, loc=mu, scale=sigma)

def q(x):
    return norm.pdf(x, loc=mu+1, scale=sigma)

def f(x):
    return norm.cdf(x, loc=mu+1, scale=sigma)

In [ ]:
# Define loss functions associated with each resilience action
def f0(x):
    return norm.cdf(x, loc=mu+1, scale=sigma)
    
def f1(x):
    return norm.cdf(x, loc=mu+1.5, scale=1.25*sigma)

def f2(x):
    return norm.cdf(x, loc=mu+1.75, scale=1.25*sigma)    

def f3(x):
    return norm.cdf(x, loc=mu+2.5, scale=1.75*sigma)

# Define constant implementation costs associated with each resilience action
cost1 = 0.15
cost2 = 0.2
cost3 = 0.5

damage_states = ['Minor Damage', 'Moderate Damage', 'Extensive Damage']

# Define cost thresholds for different damage states
threshold_minor_damage     = 0.3  
threshold_moderate_damage  = 0.5  
threshold_extensive_damage = 0.7  

In [ ]:
def integrand(x):
    return p(x) * f(x)

def get_MC_samples(num_of_samples): 
    return norm.rvs(loc=mu, scale=sigma, size=num_of_samples)

def get_MC_estimate(num_of_samples, function):         
    return 1./num_of_samples * np.sum(function)

def get_trunc_MC_samples(num_of_samples, lower_bound):
    samples = []
    while len(samples) < num_of_samples:
        sample = norm.rvs(loc=mu, scale=sigma, size=1)
        if sample >= lower_bound:
            samples.append(sample[0])
    return np.array(samples)

def get_IS_samples(num_of_samples): 
    return norm.rvs(loc=mu+1, scale=sigma, size=num_of_samples)

def get_IS_estimate(num_of_samples, samples):         
    return 1./num_of_samples * np.sum(f(samples)*p(samples)/q(samples))

def get_MCMC_samples(num_of_samples, mu, sigma, initial_sigma):

    sample = [0 for n in range(num_of_samples)]
    function = [0 for n in range(num_of_samples)]

    sample[0] = norm.rvs(loc=mu, scale=sigma, size=1)[0]    
    function[0] = f(sample[0])  

    for n in range(1, num_of_samples):          
        proposal_sigma = 0.5*((num_of_samples-n)/num_of_samples)+initial_sigma
        
        candidate_sample = np.random.normal(sample[n-1], proposal_sigma, 1)[0]
        function[n] = f(candidate_sample)  

        prob_of_previous_sample = p(sample[n-1])
        prob_of_candidate_sample = p(candidate_sample)

        prob_of_previous_given_candidate = norm.pdf(sample[n-1], loc=candidate_sample, scale=proposal_sigma)
        prob_of_candidate_given_previous = norm.pdf(candidate_sample, loc=sample[n-1], scale=proposal_sigma)

        acceptance_ratio = (function[n] * prob_of_candidate_sample * prob_of_previous_given_candidate) / (function[n-1] * prob_of_previous_sample * prob_of_candidate_given_previous)
        if random.uniform(0, 1) < acceptance_ratio:
            sample[n] = candidate_sample
        else:
            sample[n] = sample[n-1]
            function[n] = function[n-1]

    samples = sample[:]
    return samples, function

In [ ]:
true_value, _ = quad(integrand, 0, np.inf)
true_value

0.07864960352512981

In [ ]:
MC_samples = get_MC_samples(num_of_samples)
MC_f_of_x  = f(MC_samples)

MC_estimate = np.average(MC_f_of_x)
MC_error    = np.average(np.abs(f(MC_samples)-true_value)/true_value)

print('MC Estimate =', MC_estimate)
print('MC Relative Error =', MC_error)

MC Estimate = 0.07719582484644731
MC Relative Error = 1.0489021727494157


In [ ]:
trunc_MC_samples = get_trunc_MC_samples(num_of_samples, 3)
trunc_MC_f_of_x  = f(trunc_MC_samples)

trunc_MC_estimate = np.mean(trunc_MC_f_of_x)
trunc_MC_error   = np.average(np.abs(f(trunc_MC_samples)-true_value)/true_value)

print('Truncated MC Estimate =', trunc_MC_estimate)
print('TruncatedMC Relative Error =', trunc_MC_error)

Truncated MC Estimate = 0.6474962557336648
TruncatedMC Relative Error = 7.232670308716549


In [ ]:
IS_samples  = get_IS_samples(num_of_samples)
IS_f_of_x   = f(IS_samples)*p(IS_samples)/q(IS_samples)
IS_estimate = get_IS_estimate(num_of_samples, IS_samples)
IS_error    = np.average(np.abs(IS_f_of_x-true_value)/true_value)

print('IS Estimate =', IS_estimate)
print('IS Relative Error =', IS_error)

IS Estimate = 0.08608352445894887
IS Relative Error = 0.6403922671973562


In [ ]:
MCMC_samples, MCMC_f_of_x = get_MCMC_samples(num_of_MCMC_samples, mu, sigma, sigma)
kde = gaussian_kde(np.array(MCMC_samples).T, bw_method = 'silverman')

MCMC_IS_samples = kde.resample(num_of_samples)[0]
pp = p(MCMC_IS_samples)
qq = kde.evaluate(MCMC_IS_samples)
ff = f(MCMC_IS_samples)

MCMC_IS_estimate = np.average((pp/qq)*ff)
MCMC_IS_error    = np.average(np.abs((pp/qq)*ff-true_value)/true_value)

print('MCMC_IS Estimate =', MCMC_IS_estimate)
print('MCMC_IS Error =', MCMC_IS_error)

MCMC_IS Estimate = 0.07862216641859832
MCMC_IS Error = 0.04656881513294189


In [ ]:
fig = go.Figure()
fig.update_layout(
    xaxis=dict(
        showline=True, zeroline=True, linewidth=2, linecolor='black', mirror = True,
        showgrid=False, showticklabels=True, ticks="outside", tickfont=dict(size=15, family="times new roman", color='black'),
        title='Hazard Intensity', titlefont=dict(size=18, family="times new roman", color='black'),
        range=[0, 10]),
    yaxis=dict(
        showline=True, zeroline=True, linewidth=2, linecolor='black', mirror= True, side="left",
        showgrid=True, showticklabels=True, ticks="outside", tickfont_size=15,
        title='Expected Loss ($million)', titlefont=dict(size=18, family="times new roman", color='black'),  
        range=[0, 10]),
    yaxis2=dict(
        showline=True, zeroline=False, linewidth=2, linecolor='black',
        showgrid=False, showticklabels=False, 
        range=[0, 1]),
    showlegend = True,
    legend_font=dict(
        family="Times New Roman",
        size=15,
        color="black"),
    paper_bgcolor = 'rgba(0,0,0,0)',
    plot_bgcolor  = 'rgba(0,0,0,0)',
    height = 500,
    width  = 800)

fig.add_scatter(x=x, y=f0(x), line=dict(dash='solid', color='white'), name='No Resilience Action', yaxis='y', showlegend=False)
fig.add_scatter(x=x, y=f0(x), line=dict(dash='solid', color='black'), name='No Resilience Action', yaxis='y2')
fig.add_scatter(x=x, y=f1(x), line=dict(dash='dot', color='green'), name='Resilience Action 1', yaxis='y2')
fig.add_scatter(x=x, y=f2(x), line=dict(dash='dash', color='green'), name='Resilience Action 2', yaxis='y2')
fig.add_scatter(x=x, y=f3(x), line=dict(dash='dashdot', color='green'), name='Resilience Action 3', yaxis='y2')
   
fig.show()
# fig.write_image("loss_functions.png")

In [ ]:
# Sample scenarios in low probability region
low_prob_scenarios = np.random.normal(2.5, 0.5, num_of_samples)

# Sample scenarios in high probability region
high_prob_scenarios = np.random.normal(4.0, 0.5, num_of_samples)

true_scenario = [norm.ppf(true_value, loc=mu+1, scale=sigma)]

In [ ]:
# Initialize arrays to store fragility curve data for each resilience action and damage state
fragility_minor_damage_f0     = []
fragility_moderate_damage_f0  = []
fragility_extensive_damage_f0 = []

fragility_minor_damage_f1     = []
fragility_moderate_damage_f1  = []
fragility_extensive_damage_f1 = []

fragility_minor_damage_f2     = []
fragility_moderate_damage_f2  = []
fragility_extensive_damage_f2 = []

fragility_minor_damage_f3     = []
fragility_moderate_damage_f3  = []
fragility_extensive_damage_f3 = []

# Calculate fragility curves for each resilience action and damage state
for scenario in MC_samples:
    fragility_minor_damage_f0.append(int(f0(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f0.append(int(f0(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f0.append(int(f0(scenario) >= threshold_extensive_damage))

    fragility_minor_damage_f1.append(int(f1(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f1.append(int(f1(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f1.append(int(f1(scenario) >= threshold_extensive_damage))

    fragility_minor_damage_f2.append(int(f2(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f2.append(int(f2(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f2.append(int(f2(scenario) >= threshold_extensive_damage))

    fragility_minor_damage_f3.append(int(f3(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f3.append(int(f3(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f3.append(int(f3(scenario) >= threshold_extensive_damage))

# Calculate probabilities of exceedance for each damage state
probability_minor_damage_f0 = np.mean(fragility_minor_damage_f0)
probability_moderate_damage_f0 = np.mean(fragility_moderate_damage_f0)
probability_extensive_damage_f0 = np.mean(fragility_extensive_damage_f0)

probability_minor_damage_f1 = np.mean(fragility_minor_damage_f1)
probability_moderate_damage_f1 = np.mean(fragility_moderate_damage_f1)
probability_extensive_damage_f1 = np.mean(fragility_extensive_damage_f1)

probability_minor_damage_f2 = np.mean(fragility_minor_damage_f2)
probability_moderate_damage_f2 = np.mean(fragility_moderate_damage_f2)
probability_extensive_damage_f2 = np.mean(fragility_extensive_damage_f2)

probability_minor_damage_f3 = np.mean(fragility_minor_damage_f3)
probability_moderate_damage_f3 = np.mean(fragility_moderate_damage_f3)
probability_extensive_damage_f3 = np.mean(fragility_extensive_damage_f3)

MC_probabilities_f0 = [probability_minor_damage_f0, probability_moderate_damage_f0, probability_extensive_damage_f0]
MC_probabilities_f1 = [probability_minor_damage_f1, probability_moderate_damage_f1, probability_extensive_damage_f1]
MC_probabilities_f2 = [probability_minor_damage_f2, probability_moderate_damage_f2, probability_extensive_damage_f2]
MC_probabilities_f3 = [probability_minor_damage_f3, probability_moderate_damage_f3, probability_extensive_damage_f3]

In [ ]:
# Initialize arrays to store fragility curve data for each resilience action and damage state
fragility_minor_damage_f0     = []
fragility_moderate_damage_f0  = []
fragility_extensive_damage_f0 = []

fragility_minor_damage_f1     = []
fragility_moderate_damage_f1  = []
fragility_extensive_damage_f1 = []

fragility_minor_damage_f2     = []
fragility_moderate_damage_f2  = []
fragility_extensive_damage_f2 = []

fragility_minor_damage_f3     = []
fragility_moderate_damage_f3  = []
fragility_extensive_damage_f3 = []

# Calculate fragility curves for each resilience action and damage state
for scenario in trunc_MC_samples:
    fragility_minor_damage_f0.append(int(f0(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f0.append(int(f0(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f0.append(int(f0(scenario) >= threshold_extensive_damage))

    fragility_minor_damage_f1.append(int(f1(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f1.append(int(f1(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f1.append(int(f1(scenario) >= threshold_extensive_damage))

    fragility_minor_damage_f2.append(int(f2(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f2.append(int(f2(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f2.append(int(f2(scenario) >= threshold_extensive_damage))

    fragility_minor_damage_f3.append(int(f3(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f3.append(int(f3(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f3.append(int(f3(scenario) >= threshold_extensive_damage))

# Calculate probabilities of exceedance for each damage state
probability_minor_damage_f0 = np.mean(fragility_minor_damage_f0)
probability_moderate_damage_f0 = np.mean(fragility_moderate_damage_f0)
probability_extensive_damage_f0 = np.mean(fragility_extensive_damage_f0)

probability_minor_damage_f1 = np.mean(fragility_minor_damage_f1)
probability_moderate_damage_f1 = np.mean(fragility_moderate_damage_f1)
probability_extensive_damage_f1 = np.mean(fragility_extensive_damage_f1)

probability_minor_damage_f2 = np.mean(fragility_minor_damage_f2)
probability_moderate_damage_f2 = np.mean(fragility_moderate_damage_f2)
probability_extensive_damage_f2 = np.mean(fragility_extensive_damage_f2)

probability_minor_damage_f3 = np.mean(fragility_minor_damage_f3)
probability_moderate_damage_f3 = np.mean(fragility_moderate_damage_f3)
probability_extensive_damage_f3 = np.mean(fragility_extensive_damage_f3)

truncated_MC_probabilities_f0 = [probability_minor_damage_f0, probability_moderate_damage_f0, probability_extensive_damage_f0]
truncated_MC_probabilities_f1 = [probability_minor_damage_f1, probability_moderate_damage_f1, probability_extensive_damage_f1]
truncated_MC_probabilities_f2 = [probability_minor_damage_f2, probability_moderate_damage_f2, probability_extensive_damage_f2]
truncated_MC_probabilities_f3 = [probability_minor_damage_f3, probability_moderate_damage_f3, probability_extensive_damage_f3]

In [ ]:
# Initialize arrays to store fragility curve data for each resilience action and damage state
fragility_minor_damage_f0     = []
fragility_moderate_damage_f0  = []
fragility_extensive_damage_f0 = []

fragility_minor_damage_f1     = []
fragility_moderate_damage_f1  = []
fragility_extensive_damage_f1 = []

fragility_minor_damage_f2     = []
fragility_moderate_damage_f2  = []
fragility_extensive_damage_f2 = []

fragility_minor_damage_f3     = []
fragility_moderate_damage_f3  = []
fragility_extensive_damage_f3 = []

# Calculate fragility curves for each resilience action and damage state
for scenario in MCMC_IS_samples:
    fragility_minor_damage_f0.append(int(f0(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f0.append(int(f0(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f0.append(int(f0(scenario) >= threshold_extensive_damage))

    fragility_minor_damage_f1.append(int(f1(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f1.append(int(f1(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f1.append(int(f1(scenario) >= threshold_extensive_damage))

    fragility_minor_damage_f2.append(int(f2(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f2.append(int(f2(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f2.append(int(f2(scenario) >= threshold_extensive_damage))

    fragility_minor_damage_f3.append(int(f3(scenario) >= threshold_minor_damage))
    fragility_moderate_damage_f3.append(int(f3(scenario) >= threshold_moderate_damage))
    fragility_extensive_damage_f3.append(int(f3(scenario) >= threshold_extensive_damage))

# Calculate probabilities of exceedance for each damage state
probability_minor_damage_f0 = np.mean(fragility_minor_damage_f0)
probability_moderate_damage_f0 = np.mean(fragility_moderate_damage_f0)
probability_extensive_damage_f0 = np.mean(fragility_extensive_damage_f0)

probability_minor_damage_f1 = np.mean(fragility_minor_damage_f1)
probability_moderate_damage_f1 = np.mean(fragility_moderate_damage_f1)
probability_extensive_damage_f1 = np.mean(fragility_extensive_damage_f1)

probability_minor_damage_f2 = np.mean(fragility_minor_damage_f2)
probability_moderate_damage_f2 = np.mean(fragility_moderate_damage_f2)
probability_extensive_damage_f2 = np.mean(fragility_extensive_damage_f2)

probability_minor_damage_f3 = np.mean(fragility_minor_damage_f3)
probability_moderate_damage_f3 = np.mean(fragility_moderate_damage_f3)
probability_extensive_damage_f3 = np.mean(fragility_extensive_damage_f3)

MCMC_IS_probabilities_f0 = [probability_minor_damage_f0, probability_moderate_damage_f0, probability_extensive_damage_f0]
MCMC_IS_probabilities_f1 = [probability_minor_damage_f1, probability_moderate_damage_f1, probability_extensive_damage_f1]
MCMC_IS_probabilities_f2 = [probability_minor_damage_f2, probability_moderate_damage_f2, probability_extensive_damage_f2]
MCMC_IS_probabilities_f3 = [probability_minor_damage_f3, probability_moderate_damage_f3, probability_extensive_damage_f3]

In [ ]:
# Data for different simulation methods and resilience actions
methods = ['MC Hazard Scenarios', 'Truncated MC Hazard Scenarios', 'MCMC-IS Simulation']

probabilities_f0 = [MC_probabilities_f0, truncated_MC_probabilities_f0, MCMC_IS_probabilities_f0]
probabilities_f1 = [MC_probabilities_f1, truncated_MC_probabilities_f1, MCMC_IS_probabilities_f1]
probabilities_f2 = [MC_probabilities_f2, truncated_MC_probabilities_f2, MCMC_IS_probabilities_f2]
probabilities_f3 = [MC_probabilities_f3, truncated_MC_probabilities_f3, MCMC_IS_probabilities_f3]

fig = make_subplots(rows=3, cols=1, subplot_titles=methods, shared_xaxes=True)

for i in range(3):
    showlegend = i == 0  # Only show legend for the first subplot

    fig.add_trace(go.Bar(
        x=damage_states,
        y=[p * 100 for p in probabilities_f0[i]],
        name='No Resilience Action',
        marker_color='rgba(55, 83, 109, 0.7)',
        showlegend=showlegend
    ), row=i+1, col=1)
    
    fig.add_trace(go.Bar(
        x=damage_states,
        y=[p * 100 for p in probabilities_f1[i]],
        name='Resilience Action 1',
        marker_color='rgba(26, 118, 255, 0.7)',
        showlegend=showlegend
    ), row=i+1, col=1)
    
    fig.add_trace(go.Bar(
        x=damage_states,
        y=[p * 100 for p in probabilities_f2[i]],
        name='Resilience Action 2',
        marker_color='rgba(102, 204, 0, 0.7)',
        showlegend=showlegend
    ), row=i+1, col=1)
    
    fig.add_trace(go.Bar(
        x=damage_states,
        y=[p * 100 for p in probabilities_f3[i]],
        name='Resilience Action 3',
        marker_color='rgba(255, 153, 51, 0.7)',
        showlegend=showlegend
    ), row=i+1, col=1)

# Update layout
fig.update_layout(
    barmode='group',
    showlegend = True,
    legend_font=dict(
        family="Times New Roman",
        size=15,
        color="black"),
    paper_bgcolor = 'rgba(0,0,0,0)',
    plot_bgcolor  = 'rgba(0,0,0,0)',
    height = 1000,
    width  = 800)

# Update x and y axes for all subplots
fig.update_xaxes(showline=True, zeroline=True, linewidth=2, linecolor='black', mirror=True,
                 showgrid=False, showticklabels=True, ticks="outside", 
                 tickfont=dict(size=15, family="times new roman", color='black'),
                 title='', titlefont=dict(size=18, family="times new roman", color='black'))

fig.update_yaxes(showline=True, zeroline=True, linewidth=2, linecolor='black', mirror=True,
                 showgrid=True, showticklabels=True, ticks="outside", 
                 tickfont=dict(size=15, family="times new roman", color='black'),
                 range=[0, 100],
                 title='Probability of Exceedance (%)', titlefont=dict(size=15, family="times new roman", color='black'))

# Define a list of annotations for subplot titles with custom font and size
annotations = []
y = [1.03, 0.65, 0.26]
for i, method in enumerate(methods):
    annotations.append(dict(xref='paper', yref='paper', x=0.5, y=y[i],
                            xanchor='center', yanchor='top',
                            text=method,
                            font=dict(family='Times New Roman', size=15, color='black'),
                            showarrow=False))

# Add annotations to layout
fig.update_layout(annotations=annotations)               

# Show the plot
fig.show()
# fig.write_image("exceedance_probabilities.png")
fig.write_image("exceedance_probabilities.svg")

In [ ]:
benefit1, benefit2, benefit3 = [], [], []

# Cost-benefit analysis for each set of scenarios
for scenario in MC_samples:
    benefit1.append(f0(scenario)-f1(scenario)-cost1)
    benefit2.append(f0(scenario)-f2(scenario)-cost2)
    benefit3.append(f0(scenario)-f3(scenario)-cost3)

# Print or store the results for decision-making
print(f"  Resilience Action 1: Benefit={np.mean(benefit1)}")
print(f"  Resilience Action 2: Benefit={np.mean(benefit2)}")
print(f"  Resilience Action 3: Benefit={np.mean(benefit3)}")

  Resilience Action 1: Benefit=-0.10111832037246707
  Resilience Action 2: Benefit=-0.13529516353111662
  Resilience Action 3: Benefit=-0.4287400170907801


In [ ]:
benefit1, benefit2, benefit3 = [], [], []

# Cost-benefit analysis for each set of scenarios
for scenario in trunc_MC_samples:
    benefit1.append(f0(scenario)-f1(scenario)-cost1)
    benefit2.append(f0(scenario)-f2(scenario)-cost2)
    benefit3.append(f0(scenario)-f3(scenario)-cost3)

# Print or store the results for decision-making
print(f"  Resilience Action 1: Benefit={np.mean(benefit1)}")
print(f"  Resilience Action 2: Benefit={np.mean(benefit2)}")
print(f"  Resilience Action 3: Benefit={np.mean(benefit3)}")

  Resilience Action 1: Benefit=0.17600228281597555
  Resilience Action 2: Benefit=0.2494089397444874
  Resilience Action 3: Benefit=0.07478164971956347


In [ ]:
benefit1, benefit2, benefit3 = [], [], []

# Cost-benefit analysis for each set of scenarios
for scenario in IS_samples:
    benefit1.append(f0(scenario)-f1(scenario)-cost1)
    benefit2.append(f0(scenario)-f2(scenario)-cost2)
    benefit3.append(f0(scenario)-f3(scenario)-cost3)

# Print or store the results for decision-making
print(f"  Resilience Action 1: Benefit={np.mean(benefit1)}")
print(f"  Resilience Action 2: Benefit={np.mean(benefit2)}")
print(f"  Resilience Action 3: Benefit={np.mean(benefit3)}")

  Resilience Action 1: Benefit=0.06914878917948925
  Resilience Action 2: Benefit=0.10295458128031494
  Resilience Action 3: Benefit=-0.10499792192497083


In [ ]:
benefit1, benefit2, benefit3 = [], [], []

# Cost-benefit analysis for each set of scenarios
for scenario in MCMC_IS_samples:
    benefit1.append(f0(scenario)-f1(scenario)-cost1)
    benefit2.append(f0(scenario)-f2(scenario)-cost2)
    benefit3.append(f0(scenario)-f3(scenario)-cost3)

# Print or store the results for decision-making
print(f"  Resilience Action 1: Benefit={np.mean(benefit1)}")
print(f"  Resilience Action 2: Benefit={np.mean(benefit2)}")
print(f"  Resilience Action 3: Benefit={np.mean(benefit3)}")

  Resilience Action 1: Benefit=-0.0062568304999850565
  Resilience Action 2: Benefit=-0.007116525393772968
  Resilience Action 3: Benefit=-0.27140698558110765


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1769d2a2-cc69-4c7c-a48d-a5310f119005' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>